In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [0]:
#define constants
#unrolled through 100 time steps
time_steps=100
#hidden LSTM units
num_units=200
#rows of 12 sensors
n_input=12
#learning rate for adam
learning_rate=0.001
#this is meant to be classified in 6 classes{'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
n_classes=6
#size of batch
batch_size=120

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_data = pd.read_csv("/content/gdrive/My Drive/train_data.csv")
test_data = pd.read_csv("/content/gdrive/My Drive/test_data.csv")

In [0]:
# dataTemp : CSV input/out 섞여있는 데이터 
# period : 자를 timeSteps
# onehot : default True, 원핫으로 return 할지 말지 결정
# return : 다 만들어진 X , Y
def dataPreprocessing(dataTemp , period , onehot = True , dropCol = True ):
    encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
    retY = pd.Series([])
#     retY = np.array([])
    retX = dataTemp
    temptype = -1
    cnt = 0
    dataTemp["type_1"] = dataTemp["user_id"].map(str) +"_"+ dataTemp["action_id"]
    
    for dataIter in range(dataTemp["action_id"].size//period):
        retY[cnt] = encoding[dataTemp["action_id"].iloc[dataIter*period][:3]]
        comp = dataTemp["type_1"].iloc[dataIter*period]
        if comp == temptype:
            cnt += 1
        else:
#             print(cnt)
            temptype = comp
#             cnt += 1
            retX = retX.drop(retX.index[dataIter:dataIter+period])
    if onehot:
        retY = pd.get_dummies(retY)
    if dropCol:
        retX = retX.drop(["action_id"] , axis = 1)
        retX = retX.drop(["user_id"] , axis = 1)
        retX = retX.drop(["time_50hz"] , axis = 1)
        retX = retX.drop(["type_1"] , axis = 1)
    return retX , retY
        

In [0]:
# dataTemp : CSV input/out 섞여있는 데이터 
# period : 자를 timeSteps
# onehot : default True, 원핫으로 return 할지 말지 결정
# return : 다 만들어진 X , Y
def dataPreprocessingWithNormal(dataTemp , period , onehot = True , dropCol = True ):
  encoding = {'jog': 0, 'dws': 1, 'ups': 2, 'wlk': 3, 'sit': 4, 'std': 5}
  retY = pd.Series([])
#     retY = np.array([])
  retX = dataTemp
  retX_X = pd.DataFrame([])
  temptype = -1
  cnt = 0
  dataTemp["type_1"] = dataTemp["user_id"].map(str) +"_"+ dataTemp["action_id"]

#   for i in set(dataTemp["type_1"]):
  a = dataTemp
  a = a.drop(["action_id"] , axis = 1)
  a = a.drop(["user_id"] , axis = 1)
  a = a.drop(["time_50hz"] , axis = 1)
  a = a.drop(["type_1"] , axis = 1)
  normalized_df=(a-a.mean())/a.std()
  retX_X = retX_X.append( normalized_df )
#     x = a.values
#     min_max_scaler = preprocessing.MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     retX_X = retX_X.append( pd.DataFrame(x_scaled) )
  retX_X = retX_X.reset_index(drop=True)
#   print(retX_X.shape)


  for dataIter in range(dataTemp["action_id"].size//period):
    retY[cnt] = encoding[dataTemp["action_id"].iloc[dataIter*period][:3]]
    comp = dataTemp["type_1"].iloc[dataIter*period]
#     print(dataTemp.shape , retX_X.shape)
    if comp == temptype:
      cnt += 1
    elif temptype == -1 :
      cnt += 1
    else:            
      temptype = comp
      retX_X = retX_X.drop(retX_X.index[dataIter:dataIter+period])
#             print(dataTemp.shape . retX_X.shape)
  if onehot:
    retY = pd.get_dummies(retY)
#     if dropCol:
#         retX = retX.drop(["action_id"] , axis = 1)
#         retX = retX.drop(["user_id"] , axis = 1)
#         retX = retX.drop(["time_50hz"] , axis = 1)
#         retX = retX.drop(["type_1"] , axis = 1)
#   print(retX_X.shape)
  return retX_X , retY
        

In [0]:
train_X , train_Y = dataPreprocessingWithNormal(train_data , time_steps , True)
test_X , test_Y = dataPreprocessingWithNormal(test_data , time_steps , True)

In [0]:
train_X , train_Y = dataPreprocessing(train_data , time_steps , True)
test_X , test_Y = dataPreprocessing(test_data , time_steps , True)

In [0]:
tempaaa = pd.DataFrame([])
for i in set(train_data["type_1"]):
  a = train_data[(train_data["type_1"]==i)]
  a = a.drop(["action_id"] , axis = 1)
  a = a.drop(["user_id"] , axis = 1)
  a = a.drop(["time_50hz"] , axis = 1)
  a = a.drop(["type_1"] , axis = 1)
  normalized_df=(a-a.mean())/a.std()
  tempaaa = tempaaa.append( normalized_df )
tempaaa = tempaaa.reset_index(drop=True)
  
  

In [38]:
train_X.shape , train_Y.shape , test_X.shape , test_Y.shape

((1209003, 12), (12090, 6), (167862, 12), (1678, 6))

In [36]:
train_X.shape , train_Y.shape , test_X.shape , test_Y.shape

((1209003, 12), (12090, 6), (167862, 12), (1678, 6))

In [0]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder(tf.float32 ,[None,time_steps , n_input])
#input label placeholder
y=tf.placeholder(tf.int32 ,[None,n_classes])

In [0]:
input=tf.unstack(x ,time_steps,1)

In [42]:
#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=0.9)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [0]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [0]:
Yhat = tf.argmax(prediction,1)

In [0]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [49]:
#initialize variables
init=tf.global_variables_initializer()
test_X_temp = np.array(test_X[:160000]).reshape(-1,time_steps , n_input)
epoch = 50
sess = tf.Session()

sess.run(init)
for ep  in range(1,epoch+1):    
    iter=0
    while iter<99:
        batch_x = train_X[iter*batch_size*time_steps:(iter+1)*batch_size*time_steps]
        batch_y = train_Y[iter*batch_size:(iter+1)*batch_size]
        batch_x = np.array(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})  
#             if iter % 10 == 0:
#                 acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
#                 los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
        iter=iter+1
    if ep % 5 == 0:            
        test_acc = sess.run(accuracy, feed_dict={x: test_X_temp, y: test_Y[:1600]})
        test_los = sess.run(loss, feed_dict={x: test_X_temp, y: test_Y[:1600]})
        print("Epoch {} , Test Acc {} , los {} ".format(ep,test_acc , test_los))


Epoch 5 , Test Acc 0.4268749952316284 , los 2.708103656768799 
Epoch 10 , Test Acc 0.5450000166893005 , los 1.451491117477417 
Epoch 15 , Test Acc 0.7131249904632568 , los 1.2066409587860107 
Epoch 20 , Test Acc 0.7681249976158142 , los 1.0570793151855469 
Epoch 25 , Test Acc 0.7825000286102295 , los 1.0439800024032593 
Epoch 30 , Test Acc 0.7931249737739563 , los 1.081236720085144 
Epoch 35 , Test Acc 0.7881249785423279 , los 1.1663413047790527 
Epoch 40 , Test Acc 0.28125 , los 10.241960525512695 
Epoch 45 , Test Acc 0.765625 , los 1.0639581680297852 
Epoch 50 , Test Acc 0.7743750214576721 , los 1.1466714143753052 


In [0]:
# test_data_eval 에 분류를 위한 type_1 을 넣는데.
test_data_eval = test_data
test_data_eval["type_1"] = test_data["user_id"].map(str) +"_"+ test_data["action_id"]

In [24]:
# type 의 종류를 가지고 와서 나중에 loop 를 돌면서 accuracy 를 만들것임
dic_testType = list(set(test_data_eval["type_1"]))
dic_testType

['2_jog_9',
 '1_sit_5',
 '2_jog_16',
 '2_sit_5',
 '1_wlk_15',
 '1_wlk_8',
 '2_std_14',
 '2_ups_4',
 '1_jog_9',
 '1_std_6',
 '2_wlk_7',
 '2_ups_3',
 '3_wlk_8',
 '1_dws_11',
 '3_jog_16',
 '1_ups_3',
 '3_std_14',
 '3_dws_11',
 '3_dws_1',
 '1_ups_4',
 '1_dws_1',
 '1_sit_13',
 '3_dws_2',
 '1_wlk_7',
 '3_ups_3',
 '2_wlk_8',
 '3_ups_12',
 '2_wlk_15',
 '1_dws_2',
 '1_std_14',
 '2_dws_2',
 '1_ups_12',
 '3_sit_13',
 '3_jog_9',
 '3_std_6',
 '3_wlk_7',
 '2_std_6',
 '2_ups_12',
 '3_ups_4',
 '1_jog_16',
 '3_sit_5',
 '2_sit_13',
 '3_wlk_15',
 '2_dws_11',
 '2_dws_1']

In [23]:
# full loop 만들어보기
y_pred = []
y_true = []
# >>> accuracy_score(y_true, y_pred)
for ite in dic_testType:
    # 한 타입만 일단 가져와보고
    tempX = test_data_eval[(test_data_eval["type_1"] == ite)]
    # matrix 에 맞게 뒤에 잘라내줌.
    tempX = tempX[:(tempX.shape[0] // time_steps)*time_steps]
    # 여기에서 데이터를 X,Y 로 먼저 나눠준다. onehot 해뒀고 필요없는 column 까지 다 Drop
    evalTestX , evalTestY = dataPreprocessing(tempX , time_steps , True , True)
    evalTestX = np.array(evalTestX).reshape(-1,time_steps , n_input)
    # one hot 에 하나만 있기때문에 나머지 다 넣어주고 순서 원래대로 변경
#         for i in range(6):
#             if i not in set(evalTestY):
#                 evalTestY[i] = 0
#         evalTestY=evalTestY[[0,1,2,3,4,5]]
    yh = sess.run(Yhat , feed_dict={x:evalTestX})
    pre = Counter(yh).most_common(1)[0][0]
    tru = evalTestY.columns[0]
    y_pred.append(pre)
    y_true.append(tru)
    if(pre != tru):
      print("ite : " , ite , ' predict : ' , yh ,":" ,pre, " Real Y : " , tru)
    
#     print("ite : " , ite , ' predict : ' , yh ,":" ,pre, " Real Y : " , tru)
#     print('predict : ' , pre , " Real Y : " , tru)
#     print(evalTestX.shape , evalTestY.shape , ite)}

ite :  2_ups_3  predict :  [0 2 0 2 0 2 0 1] : 0  Real Y :  2
ite :  1_ups_3  predict :  [3 3 2 3 3 2 3 3 3 2 3 3 2 2 3 3 2 3] : 3  Real Y :  2
ite :  1_ups_4  predict :  [3 2 2 3 3 2 3 3 2 3 3 1 2 3 5 2 3 3 3 2 2 3 3 2 3] : 3  Real Y :  2
ite :  1_std_14  predict :  [4 3 3 3 3 0 4 3 3 3 3 0 4 3 3 3 3 3 4 3 3 3 3 3 3 4 3 3 3 3 3] : 3  Real Y :  5
ite :  1_ups_12  predict :  [1 2 3 3 3 3 3 3] : 3  Real Y :  2
ite :  2_ups_12  predict :  [0 2] : 0  Real Y :  2


In [21]:
accuracy_score(y_true, y_pred)

0.8666666666666667